In [1]:
# 필수 라이브러리 임포트
from typing import Tuple
import matplotlib
import meep as mp
import nlopt
import numpy as np
import os

# matplotlib의 백엔드를 "agg"로 설정하여 이미지를 파일로 저장 가능하게 합니다.
matplotlib.use("agg")
import matplotlib.pyplot as plt
 
# Lorentzian 함수 정의: Lorentzian 파라미터를 사용하여 복소 유전율 프로파일을 계산
def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    N = len(p) // 3  # 파라미터가 3개씩 묶인 Lorentzian 항의 개수
    y = np.zeros(len(x))  # 결과값을 저장할 배열
    for n in range(N):
        A_n = p[3 * n + 0]  # 첫 번째 파라미터 (진폭)
        x_n = p[3 * n + 1]  # 두 번째 파라미터 (중심 주파수)
        g_n = p[3 * n + 2]  # 세 번째 파라미터 (감쇠율)
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수 계산
    return y

# 잔차 함수 정의: 실제 값과 Lorentzian 모델 간의 차이를 계산하고 그라디언트도 반환
def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray, grad: np.ndarray) -> float:
    N = len(p) // 3  # Lorentzian 항의 개수
    yp = lorentzfunc(p, x)  # 예측된 유전율 프로파일 계산
    val = np.sum(np.square(abs(y - yp)))  # 실제 값과 예측 값의 차이(L2 norm)

    # 그라디언트 계산
    for n in range(N):
        A_n = p[3 * n + 0]
        x_n = p[3 * n + 1]
        g_n = p[3 * n + 2]
        d = 1 / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수의 도함수
        if grad.size > 0:
            grad[3 * n + 0] = 2 * np.real(np.dot(np.conj(yp - y), d))  # 진폭에 대한 그라디언트
            grad[3 * n + 1] = -4 * x_n * A_n * np.real(np.dot(np.conj(yp - y), np.square(d)))  # 중심 주파수에 대한 그라디언트
            grad[3 * n + 2] = -2 * A_n * np.imag(np.dot(np.conj(yp - y), x * np.square(d)))  # 감쇠율에 대한 그라디언트
    return val  # 최종 오차 반환

# Lorentzian 파라미터 최적화 함수 정의
def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    alg=nlopt.LD_LBFGS,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    # NLopt 최적화 설정
    opt = nlopt.opt(alg, len(p0))  # 최적화 알고리즘 설정 (LD_LBFGS는 LBFGS 알고리즘)
    opt.set_ftol_rel(tol)  # 상대 오차 허용 범위 설정
    opt.set_maxeval(maxeval)  # 최대 반복 횟수 설정
    opt.set_lower_bounds(np.zeros(len(p0)))  # 하한값 설정
    opt.set_upper_bounds(float("inf") * np.ones(len(p0)))  # 상한값 설정
    opt.set_min_objective(lambda p, grad: lorentzerr(p, x, y, grad))  # 목표 함수 설정
    local_opt = nlopt.opt(nlopt.LD_LBFGS, len(p0))  # 로컬 최적화 설정
    local_opt.set_ftol_rel(1e-10)  # 로컬 최적화 상대 오차 설정
    local_opt.set_xtol_rel(1e-8)  # 로컬 최적화 X 변화 상대 오차 설정
    opt.set_local_optimizer(local_opt)  # 로컬 최적화 설정 추가
    popt = opt.optimize(p0)  # 최적화 실행
    minf = opt.last_optimum_value()  # 마지막 최적화 값 반환
    return popt, minf  # 최적화된 파라미터와 최소 오차 반환

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meep as mp
import nlopt
import os

# --- 1) 원본 permittivity 데이터 로드 (μm 단위) ---
csv_path      = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/Material data/aNDP_material_data.csv"
data          = np.genfromtxt(csv_path, delimiter=",")
wl            = data[:, 0]                    # 파장 (μm)
eps_total     = data[:, 1] + 1j*data[:, 2]     # 이미 permittivity

# --- 2) 비교용 CSV 파일 로드 (same 형식: wl, Re(eps), Im(eps)) ---
compare_path  = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/New fitting data/aNDP/lumerical_aNDP_eps.csv"
cmp_data      = np.genfromtxt(compare_path, delimiter=",")
wl_cmp        = cmp_data[:, 0]
eps_cmp       = cmp_data[:, 1] + 1j*cmp_data[:, 2]

# --- 3) 피팅 범위 설정 ---
# --- 공통 설정 (위에서 로드한 wl_red, freqs_red, eps_fit_target_raw 등 사용) ---
wl_min, wl_max = 0.39, 0.73
mask = (wl >= wl_min) & (wl <= wl_max)
wl_red    = wl[mask]
freqs_red = 1.0 / wl_red
eps_total_red = eps_total[mask]
eps_fit_target_raw = eps_total_red   # eps_inf 빼기 전 원본

import numpy as np
import meep as mp
import nlopt
from typing import Tuple
import pickle

# lorentzfit 함수는 이미 정의되어 있어야 함 (여기선 너가 가지고 있다고 가정함)

def fit_material_from_csv(
    csv_path: str,
    wl_range_nm = [0.38, 0.72],
    eps_inf: float = 1.1,
    num_lorentzians: int = 3,
    num_repeat: int = 100,
    opt_tol: float = 1e-25,
    maxeval: int = 50000,
) -> mp.Medium:
    # --- 0) 피클 경로 설정 ---
    base, _ = os.path.splitext(csv_path)
    pickle_path = base + "_fit.pkl"

    # --- 1) 피클이 있으면 로드 후 반환 ---
    if os.path.exists(pickle_path):
        with open(pickle_path, "rb") as f:
            eps_inf, E_sus, freq_range = pickle.load(f)
        print(f"Loaded fit from pickle: {pickle_path}")
        return eps_inf, E_sus, freq_range
    
    # 데이터 불러오기
    mydata = np.genfromtxt(csv_path, delimiter=",", skip_header=1)
    wl = mydata[:, 0]  # um
    n_complex = mydata[:, 1] + 1j * mydata[:, 2]
    eps_data = np.square(n_complex) - eps_inf  # 유전율에서 eps_inf 보정

    # 파장 범위 필터링
    wl_min, wl_max = wl_range_nm
    idx_start = np.where(wl > wl_min)[0][0]
    idx_end = np.where(wl < wl_max)[0][-1] + 1
    wl_reduced = wl[idx_start:idx_end]
    eps_reduced = eps_data[idx_start:idx_end]
    freqs_reduced = 1 / wl_reduced  # nm → 1/μm (Meep 단위)

    # 최적화 반복
    ps = np.zeros((num_repeat, 3 * num_lorentzians))
    mins = np.zeros(num_repeat)

    for m in range(num_repeat):
        p_rand = [10 ** (np.random.random()) for _ in range(3 * num_lorentzians)]
        ps[m, :], mins[m] = lorentzfit(
            p_rand, freqs_reduced, eps_reduced, nlopt.LD_MMA, opt_tol, maxeval
        )
        print(f"[{m:2d}] RMS error: {mins[m]:.6e} — params: {ps[m, :]}")

    # 최적 피팅 결과 선택
    idx_opt = np.argmin(mins)
    popt = ps[idx_opt, :]
    print(f"\n>> Best fit RMS error = {mins[idx_opt]:.6e}")
    print(f">> Optimal parameters = {popt}")

    # mp.Medium 구성
    E_sus = []

    for i in range(num_lorentzians):
        sigma = popt[3 * i + 0]
        freq = popt[3 * i + 1]
        gamma = popt[3 * i + 2]

        if freq == 0:
            # Drude 항
            E_sus.append(mp.DrudeSusceptibility(frequency=1.0, gamma=gamma, sigma=sigma))
        else:
            # Lorentz 항
            sigma_adj = sigma / freq**2
            E_sus.append(mp.LorentzianSusceptibility(frequency=freq, gamma=gamma, sigma=sigma_adj))

    # 유효 주파수 범위 설정
    freq_min = 1 / wl_max
    freq_max = 1 / wl_min
    freq_range = mp.FreqRange(min=freq_min, max=freq_max)

    # --- 6) 결과를 피클로 저장 ---
    with open(pickle_path, "wb") as f:
        pickle.dump((eps_inf, E_sus, freq_range), f)

    return eps_inf, E_sus, freq_range

sio2_data_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/Refractive Index data/Sio2/Material_merged.csv"
al_data_path   = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/Refractive Index data/Al/Material_merged.csv"
andp_data_path = "/home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/Refractive Index data/aNDP/mat/Material_merged.csv"

eps_inf_sio2, suscept_sio2, freq_sio2 = fit_material_from_csv(csv_path=sio2_data_path, num_lorentzians=2,  wl_range_nm = [0.38, 0.72])
eps_inf_al, suscept_al, freq_al = fit_material_from_csv(csv_path=al_data_path, num_lorentzians=7)
eps_inf_andp, suscept_andp, freq_andp = fit_material_from_csv(csv_path=andp_data_path, num_lorentzians=5,  wl_range_nm = [0.43, 0.58])

# from meep.materials import SiO2
material = mp.Medium(epsilon=eps_inf_andp, E_susceptibilities=suscept_andp)
# material = SiO2

# 새 모델 커브 계산
eps_model_new = np.array([material.epsilon(f)[0][0] for f in freqs_red])

# --- 7) 플롯 그리기 ---
fig, axes = plt.subplots(ncols=2, figsize=(10,4))

# 레이블 폰트 크기
label_fs = 20  
# 범례 폰트 크기
legend_fs = 14

# Re(ε) 플롯: 원본, Meep fit, 비교 CSV
axes[0].plot(wl_red, np.real(eps_total[mask]), 'gs', label="Material data")
axes[0].plot(wl_red, np.real(eps_model_new),    'bo',  label="Meep fit")
axes[0].plot(wl_cmp, np.real(eps_cmp),      'r-',  label="Lumerical fit")
axes[0].set_xlabel("Wavelength (μm)", fontsize=label_fs)
axes[0].set_ylabel("Re(ε)",             fontsize=label_fs)
axes[0].grid(True)
# axes[0].set( ylim=(0, 1))
axes[0].legend(fontsize=legend_fs)

# Im(ε) 플롯: 원본, Meep fit, 비교 CSV
axes[1].plot(wl_red, np.imag(eps_total[mask]), 'gs', label="Material data")
axes[1].plot(wl_red, np.imag(eps_model_new),       'bo',  label="Meep fit")
axes[1].plot(wl_cmp, np.imag(eps_cmp),         'r-',  label="Lumerical fit")
axes[1].set_xlabel("Wavelength (μm)", fontsize=label_fs)
axes[1].set_ylabel("Im(ε)",             fontsize=label_fs)
axes[1].grid(True)
# axes[1].set( ylim=(-0.1, 1))
axes[1].legend(fontsize=legend_fs)
fig.suptitle("Drude–Lorentzian Fit(sio2) - Meep vs Lumerical", fontsize=label_fs)

# --- 8) 플롯 저장 ---
out_fname = "eps_fit_with_compare1.png"
fig.savefig(out_fname, dpi=150, bbox_inches="tight")
print("Saved:", os.path.abspath(out_fname))

Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/Refractive Index data/Sio2/Material_merged_fit.pkl
Loaded fit from pickle: /home/min/EIDL/Tool/Meep/LGD/Meep code/OLED structure/Layer by Layer check/New fitting/Refractive Index data/Al/Material_merged_fit.pkl
[ 0] RMS error: 7.931981e-02 — params: [2.64176381e+00 4.58291271e+00 0.00000000e+00 6.46467416e-01
 2.65751821e+00 2.09150996e-01 6.14940361e+00 6.39568954e+00
 0.00000000e+00 9.03852192e+00 4.39733019e+00 2.55672520e-02
 3.06524663e+01 6.50998106e+00 0.00000000e+00 2.77922501e+00
 4.58188885e+00 0.00000000e+00 2.91274516e+00 4.58290421e+00
 0.00000000e+00]
[ 1] RMS error: 7.888959e-02 — params: [ 4.64853308  4.74075376  0.          0.63736033  2.65667615  0.20741785
  3.99845006  4.80375703  0.         30.16583907  7.19210309  0.
  3.09346665  4.69223853  0.          4.62346289  4.80386742  0.
  8.64051944  4.47009414  0.04462415]
[ 2] RMS error: 7.748287e-02 — params

/tmp/ipykernel_568430/3256803446.py:21: RuntimeWarning: invalid value encountered in divide
  y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수 계산
/tmp/ipykernel_568430/3256803446.py:35: RuntimeWarning: invalid value encountered in divide
  d = 1 / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수의 도함수


[63] RMS error: 7.641665e-02 — params: [ 8.57916581  5.14243839  0.          2.84666154  4.90789302  0.
  3.46008983  5.04330055  0.          5.93267434  5.10532075  0.
  0.65580084  2.65883114  0.20999832  6.04220011  4.88052476  0.03457113
 19.14011299  5.32920834  0.        ]
[64] RMS error: 7.627050e-02 — params: [3.59734333e+00 4.93969235e+00 0.00000000e+00 6.30086831e-01
 2.65637924e+00 2.05338498e-01 1.26697768e+00 4.59720655e+00
 4.97603574e-01 0.00000000e+00 6.72691932e+00 5.53519077e-39
 2.77586971e+01 5.18688972e+00 0.00000000e+00 5.19275686e+00
 4.70990693e+00 0.00000000e+00 6.96207139e+00 5.18672203e+00
 0.00000000e+00]
[65] RMS error: 7.623571e-02 — params: [ 0.          0.03272376  2.42064067 13.05004518  4.82180424  0.
  0.          1.6807562   4.84387251 19.25993505  5.22319054  0.
  0.51208654  4.63867932  1.0939132  12.00508533  5.22304062  0.
  0.63688085  2.65694642  0.2069451 ]
[66] RMS error: 7.831661e-02 — params: [ 4.70720072  4.64441107  0.          7.041924  

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# --- 1. Lumerical 데이터 전처리: 중복 파장 평균 처리 ---
df_cmp = pd.DataFrame({
    "wl": wl_cmp,
    "real": np.real(eps_cmp),
    "imag": np.imag(eps_cmp)
})

# 중복 파장을 평균내어 정리
df_cmp_avg = df_cmp.groupby("wl").mean().reset_index()
wl_cmp_unique = df_cmp_avg["wl"].values
eps_cmp_unique = df_cmp_avg["real"].values + 1j * df_cmp_avg["imag"].values

# --- 2. Meep fitting 결과 보간 함수 ---
interp_meep_real = interp1d(wl_red, np.real(eps_model_new), kind='cubic', bounds_error=False, fill_value="extrapolate")
interp_meep_imag = interp1d(wl_red, np.imag(eps_model_new), kind='cubic', bounds_error=False, fill_value="extrapolate")

# --- 3. Lumerical 보간 함수 ---
interp_cmp_real = interp1d(wl_cmp_unique, np.real(eps_cmp_unique), kind='cubic', bounds_error=False, fill_value="extrapolate")
interp_cmp_imag = interp1d(wl_cmp_unique, np.imag(eps_cmp_unique), kind='cubic', bounds_error=False, fill_value="extrapolate")

# --- 4. 공통 파장 축 생성 및 보간 ---
wl_common = np.linspace(max(wl_red.min(), wl_cmp_unique.min()), min(wl_red.max(), wl_cmp_unique.max()), 300)

eps_meep_real = interp_meep_real(wl_common)
eps_meep_imag = interp_meep_imag(wl_common)
eps_cmp_real  = interp_cmp_real(wl_common)
eps_cmp_imag  = interp_cmp_imag(wl_common)

# 절대 오차 계산
abs_err_real = np.abs(eps_cmp_real - eps_meep_real)
abs_err_imag = np.abs(eps_cmp_imag - eps_meep_imag)

# 절대 오차 플롯 (log scale 없음)
plt.figure(figsize=(10, 4))

# Re(ε)
plt.subplot(1, 2, 1)
plt.plot(wl_common, abs_err_real, 'b')
plt.xlabel("Wavelength (μm)", fontsize=label_fs)
# plt.ylabel("Absolute Error Re(ε)", fontsize=label_fs)
plt.title("Absolute Error Re(ε)", fontsize=label_fs)
plt.grid(True)

# Im(ε)
plt.subplot(1, 2, 2)
plt.plot(wl_common, abs_err_imag, 'r')
plt.xlabel("Wavelength (μm)", fontsize=label_fs)
# plt.ylabel("Absolute Error Im(ε)", fontsize=label_fs)
plt.title("Absolute Error Im(ε)", fontsize=label_fs)
plt.grid(True)

plt.tight_layout()
plt.savefig("absolute_error_only.png", dpi=150)
plt.show()


/tmp/ipykernel_568430/678144480.py:59: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
